In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from setsloaders import create_datasets, create_loaders, create_loaders_umap, draw_umap
from umap import UMAP

# append the filepath to where torch is installed
sys.path.append('/home/username/.local/lib/python3.10/site-packages')
# sys.path.append('/home/millerm/.local/lib/python3.10/site-packages')

import torch
import torch.nn as nn
import torch.optim as optim

## Load and Preprocess Data

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/mitbih_train.csv"
df_train = pd.read_csv(file_name,header=None)
x_train = df_train.iloc[:, df_train.columns != 187]
x_train = x_train.values.reshape(-1, 1, 187)
train_target = df_train.iloc[:, 187]
train_target = train_target.values
np.unique(train_target,return_counts=True)

(array([0., 1., 2., 3., 4.]), array([72471,  2223,  5788,   641,  6431]))

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/mitbih_test.csv"
df_test = pd.read_csv(file_name,header=None)
x_test = df_test.iloc[:, df_test.columns != 187]
x_test = x_test.values.reshape(-1, 1, 187)
test_target = df_test.iloc[:, 187]
test_target = test_target.values
np.unique(test_target,return_counts=True)

(array([0., 1., 2., 3., 4.]), array([18118,   556,  1448,   162,  1608]))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
datasets = create_datasets(x_train, x_test, train_target, test_target, seed=123)
trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

## Load and Train Model

In [ ]:
input_channels = 1
num_classes = 5
train_loader = trn_dl
test_loader = tst_dl
num_epochs = 30
learning_rate = 0.005

In [ ]:
class cnn(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3)
        self.fc1 = nn.Linear(5376, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

    def forward(self, x, return_embedding=False):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)

        embedding = self.relu(self.fc1(x))

        if return_embedding:
            return embedding

        x = self.fc2(embedding)
        return x

model = cnn(input_channels, num_classes).to(device)

In [ ]:
model = torch.load("models/vanillacnn.pth")
model = model.to(device)

In [ ]:
model.fc2 = nn.Linear(128, 5, bias=True).to(device)

In [ ]:
print(model)

cnn(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=5376, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
  (relu): ReLU()
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss}")

model.eval()
predictions = []
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(predicted,labels)
        predicted = predicted.to("cpu")
        predictions = np.append(predictions,predicted)

    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

Epoch: 0, Loss: 0.049815043807029724
Epoch: 1, Loss: 0.060355279594659805
Epoch: 2, Loss: 0.04427352547645569
Epoch: 3, Loss: 0.04481777548789978
Epoch: 4, Loss: 0.021129000931978226
Epoch: 5, Loss: 0.03254224359989166
Epoch: 6, Loss: 0.04945102706551552
Epoch: 7, Loss: 0.03206166625022888
Epoch: 8, Loss: 0.0032818750478327274
Epoch: 9, Loss: 0.040182117372751236
Epoch: 10, Loss: 0.009713441133499146
Epoch: 11, Loss: 0.012603596784174442
Epoch: 12, Loss: 0.0328601598739624
Epoch: 13, Loss: 0.008240501396358013
Epoch: 14, Loss: 0.03278488665819168
Epoch: 15, Loss: 0.007132796570658684
Epoch: 16, Loss: 0.0127171715721488
Epoch: 17, Loss: 0.004398918244987726
Epoch: 18, Loss: 0.013920842669904232
Epoch: 19, Loss: 0.013094858266413212
Epoch: 20, Loss: 0.017962563782930374
Epoch: 21, Loss: 0.005441781599074602
Epoch: 22, Loss: 0.008679995313286781
Epoch: 23, Loss: 0.01496509462594986
Epoch: 24, Loss: 0.01852482743561268
Epoch: 25, Loss: 0.05827183276414871
Epoch: 26, Loss: 0.011026183143258

In [ ]:
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())

predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

precision = precision_score(ground_truth, predictions, average="macro")
recall = recall_score(ground_truth, predictions, average="macro")
f1 = f1_score(ground_truth, predictions, average="macro")

print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))

Precision: 0.91
Recall: 0.89
F1-Score: 0.90


In [ ]:
torch.save(model, "models/transfercnn_finetune.pth")

## Create Dataloaders

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_train.csv"
df_train = pd.read_csv(file_name,header=None)
x_train = df_train.iloc[:, df_train.columns != 187]
x_train_ptb = x_train.values.reshape(-1, 1, 187)
train_target = df_train.iloc[:, 187]
train_target_ptb = train_target.values
train_target_ptb

array([1., 1., 0., ..., 1., 1., 1.])

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_test.csv"
df_test = pd.read_csv(file_name,header=None)
x_test = df_test.iloc[:, df_test.columns != 187]
x_test_ptb = x_test.values.reshape(-1, 1, 187)
test_target = df_test.iloc[:, 187]
test_target_ptb = test_target.values
test_target_ptb

array([0., 1., 0., ..., 1., 1., 0.])

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/mitbih_train.csv"
df_train = pd.read_csv(file_name,header=None)
x_train = df_train.iloc[:, df_train.columns != 187]
x_train_mit = x_train.values.reshape(-1, 1, 187)
train_target = df_train.iloc[:, 187]
train_target_mit = train_target.values
train_target_mit

array([0., 0., 0., ..., 4., 4., 4.])

In [ ]:
file_name = "ml4h_data/project2/project2_TS_input/mitbih_test.csv"
df_test = pd.read_csv(file_name,header=None)
x_test = df_test.iloc[:, df_test.columns != 187]
x_test_mit = x_test.values.reshape(-1, 1, 187)
test_target = df_test.iloc[:, 187]
test_target_mit = test_target.values
test_target_mit

array([0., 0., 0., ..., 4., 4., 4.])

In [ ]:
datasets_ptb = create_datasets(x_train_ptb, x_test_ptb, train_target_ptb, test_target_ptb, seed=123)
trn_dl_ptb, val_dl_ptb, tst_dl_ptb = create_loaders_umap(datasets_ptb, bs=128)

datasets_mit = create_datasets(x_train_mit, x_test_mit, train_target_mit, test_target_mit, seed=123)
trn_dl_mit, val_dl_mit, tst_dl_mit = create_loaders_umap(datasets_mit, bs=128)

In [ ]:
train_loader_ptb = trn_dl_ptb
test_loader_ptb = tst_dl_ptb
train_loader_mit = trn_dl_mit
test_loader_mit = tst_dl_mit

In [ ]:
palette = {0: "red",
           1: "blue",
           2: "green",
           3: "yellow",
           4: "purple"
          }
colors_ptb_train = [palette[label] for label in train_target_ptb]
colors_mit_train = [palette[label] for label in train_target_mit]
colors_ptb_test = [palette[label] for label in test_target_ptb]
colors_mit_test = [palette[label] for label in test_target_mit]

## Fine-Tune Model

In [ ]:
model = torch.load("models/transfercnn_finetune.pth")
model_ptb = model.to(device)
model_mit = model.to(device)
model_ptb.fc2 = nn.Linear(128, 2, bias=True).to(device)

In [ ]:
ptb_train = torch.tensor(x_train_ptb).float().to(device)
ptb_test = torch.tensor(x_test_ptb).float().to(device)
mit_train = torch.tensor(x_train_mit).float().to(device)
split = int(mit_train.shape[0] / 2)
mit_train_1 = mit_train[:split, :, :]
mit_train_2 = mit_train[split:, :, :]
mit_test = torch.tensor(x_test_mit).float().to(device)

model_ptb.eval()
with torch.no_grad():
    ptb_train_embeddings = model_ptb(ptb_train, return_embedding=True).cpu().numpy()
    ptb_test_embeddings = model_ptb(ptb_test, return_embedding=True).cpu().numpy()
model_mit.eval()
with torch.no_grad():
    mit_train_embeddings_1 = model_mit(mit_train_1, return_embedding=True).cpu().numpy()
    mit_train_embeddings_2 = model_mit(mit_train_2, return_embedding=True).cpu().numpy()
    mit_test_embeddings = model_mit(mit_test, return_embedding=True).cpu().numpy()

mit_train_embeddings = np.vstack((mit_train_embeddings_1, mit_train_embeddings_2))

In [ ]:
np.save('data/ptb_train_embeddings.npy', ptb_train_embeddings)
np.save('data/ptb_test_embeddings.npy', ptb_test_embeddings)
np.save('data/mit_train_embeddings.npy', mit_train_embeddings)
np.save('data/mit_test_embeddings.npy', mit_test_embeddings)

In [ ]:
def draw_umap(embeddings, colors, name_ds, train, s = 2, alpha = 0.5, random_state = 42):
    n_neighbors_values = [5, 15, 35]
    min_dist_values = [0.1, 0.5, 0.9]

    fig, axs = plt.subplots(3, 3, figsize=(18, 18))
    fig.suptitle(name_ds)

    for i, n_neighbors in enumerate(n_neighbors_values):
        for j, min_dist in enumerate(min_dist_values):
            emb = UMAP(n_neighbors = n_neighbors, min_dist = min_dist, random_state = random_state).fit_transform(embeddings)

            axs[i, j].scatter(emb[:, 0], emb[:, 1], c = colors, s = s, alpha = alpha)
            axs[i, j].set_title('{}: n_neighbors={}, min_dist={}'.format(name_ds, n_neighbors, min_dist))
            axs[i, j].set_xlabel('Dimension 1')
            axs[i, j].set_ylabel('Dimension 2')
            print('n_neighbors={}, min_dist={}'.format(n_neighbors, min_dist))

    if train == True:
        train = "train"
    else:
        train = "test"

    plt.tight_layout()
    plt.savefig(f"plots/umap_{name_ds}_{train}.png")
    plt.close()

In [ ]:
mit_train_embeddings = np.load('data/mit_train_embeddings.npy')
mit_test_embeddings = np.load('data/mit_test_embeddings.npy')
ptb_train_embeddings = np.load('data/ptb_train_embeddings.npy')
ptb_test_embeddings = np.load('data/ptb_test_embeddings.npy')

In [ ]:
draw_umap(ptb_test_embeddings, colors_ptb_test, "PTB", False)

/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


n_neighbors=5, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

n_neighbors=35, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.9


In [ ]:
draw_umap(ptb_train_embeddings, colors_ptb_train, "PTB", True)

/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

n_neighbors=35, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.9


In [ ]:
draw_umap(mit_train_embeddings, colors_mit_train, "MIT", True)

/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

n_neighbors=35, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.9


In [ ]:
draw_umap(mit_test_embeddings, colors_mit_test, "MIT", False)

/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=5, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

n_neighbors=35, min_dist=0.1


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.5


/cluster/courses/ml4h/jupyter/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


n_neighbors=35, min_dist=0.9
